In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
df = pd.read_csv("../data/trainset.csv", index_col="Region_Name")
dfr = df.T
dfr.rename(columns={"Region_Name":"Month"}, inplace=True)
dfr.insert(0, "Months", np.arange(1, 73))
dfr.reset_index(inplace=True)
dfr.drop(columns=["index"], inplace=True)

In [4]:
def prepare_data(timeseries_data, n_features):
	X, y =[],[ ]
	for i in range(len(timeseries_data)):
		# find the end of this pattern
		end_ix = i + n_features
		# check if we are beyond the sequence
		if end_ix > len(timeseries_data)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)


In [5]:
dict = {"Month":np.arange(73, 88)}
fincsv = pd.DataFrame(dict)
fincsv.set_index("Month")
# n_seq = 3
n_steps = 12
n_features = 1

In [6]:
model = Sequential()
# model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
# model.add(TimeDistributed(MaxPooling1D(pool_size=3)))
# model.add(TimeDistributed(Flatten()))
model.add((LSTM(256, activation='tanh', return_sequences=True, input_shape=(n_steps, n_features))))
model.add((LSTM(128, activation='tanh', return_sequences=True)))
model.add((LSTM(128, activation='tanh')))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [7]:
cor = dfr.iloc[:,1:].corr(method = "pearson")
corr = np.array(cor)
cor.shape

(3915, 3915)

In [8]:
len(cor.iloc[0, :])
cor

Region_Name,Region 1,Region 2,Region 3,Region 4,Region 5,Region 6,Region 7,Region 8,Region 9,Region 10,...,Region 3906,Region 3907,Region 3908,Region 3909,Region 3910,Region 3911,Region 3912,Region 3913,Region 3914,Region 3915
Region_Name,,,,,,,,,,,,,,,,,,,,,
Region 1,1.000000,0.964116,0.913280,0.798888,0.558123,0.911741,0.855471,0.902243,0.921137,0.894583,...,0.775192,0.735983,0.765062,0.760981,0.739141,0.175763,0.715788,0.645257,0.308479,0.713754
Region 2,0.964116,1.000000,0.947926,0.838048,0.602468,0.915952,0.848874,0.914656,0.930778,0.902358,...,0.741308,0.696367,0.720453,0.733502,0.706562,0.168383,0.693481,0.593155,0.315754,0.681085
Region 3,0.913280,0.947926,1.000000,0.897151,0.690946,0.845841,0.882829,0.913632,0.917425,0.860198,...,0.754026,0.709931,0.731435,0.737747,0.719642,0.176442,0.721161,0.634135,0.370111,0.692067
Region 4,0.798888,0.838048,0.897151,1.000000,0.830165,0.764907,0.731936,0.843886,0.809717,0.770117,...,0.601680,0.538553,0.582853,0.548783,0.651396,0.007991,0.544889,0.429847,0.238379,0.540367
Region 5,0.558123,0.602468,0.690946,0.830165,1.000000,0.482620,0.580413,0.557373,0.634400,0.475249,...,0.353684,0.260800,0.316623,0.299207,0.469569,-0.007194,0.340526,0.195690,0.235396,0.289570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Region 3911,0.175763,0.168383,0.176442,0.007991,-0.007194,0.105935,0.328747,0.109165,0.213595,0.092874,...,0.297220,0.446304,0.376086,0.343535,0.204004,1.000000,0.478096,0.487527,0.835732,0.449831
Region 3912,0.715788,0.693481,0.721161,0.544889,0.340526,0.675166,0.783984,0.686438,0.658666,0.683298,...,0.935840,0.918464,0.944015,0.960446,0.834455,0.478096,1.000000,0.908284,0.636249,0.921599
Region 3913,0.645257,0.593155,0.634135,0.429847,0.195690,0.573927,0.690799,0.588865,0.590524,0.601213,...,0.873088,0.963699,0.916978,0.871041,0.766913,0.487527,0.908284,1.000000,0.639235,0.952333


In [9]:
ind = [False]*len(cor.iloc[0, :])

# 1 2 3 -> 1,2 - 0.91 && 1,3 - 0.91 && 2,3 - 0.85

def find_corr_cols(l_thres, u_thresh):
    # ind = [False]*len(cor.iloc[0, :])
    # thresh = 0.9
    related = []
    unrelated = []
    # flag=1
    for i in range(0, len(cor.iloc[0, :])):
        if ind[i] == False:
            list = []
            list.append(i+1)
            for j in range(i+1, len(cor.iloc[0, :])):
                if l_thres < corr[i, j] and corr[i, j] <= u_thresh and ind[j] == False:
                    list.append(j+1)
                    # flag=0
                    ind[i] = True
                    ind[j] = True
                    # print("related is ",i+1)
            if len(list) == 1:
                # print("unrelated is ",i+1)
                unrelated.append(i+1)
                # ind[i] = False
            else:
                # print("related is ",i+1)
                related.append(list)
            # if flag == 1 : 
                

    return related, unrelated



related9, unrelated9 = find_corr_cols(0.85, 1)
related8, unrelated8 = find_corr_cols(0.75, 0.85)
related7, unrelated7 = find_corr_cols(0.65, 0.75)
# unrelated = unrelated8
# print(len(related9)+len(related8)+len(related7))


In [10]:
print(len(related9))
print(len(related8))
print(len(unrelated9))
s = 0

for i in range(len(related9)):
    s = s + len(related9[i])

for i in range(len(related8)):
    s = s + len(related8[i])

for i in range(len(related7)):
    s = s + len(related7[i])
print(3915 - s)



# np.array(related9[0])

450
82
378
103


In [13]:
allrelated = related9 + related8 + related7
for uns in unrelated7:
    allrelated.append([uns])

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_28648/1434738475.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  allrelated = np.array(allrelated)


(672,)

In [ ]:
# ls_correg = [[2, 12]]
ls_correg = allrelated

for pee in allrelated:
    region = "Region "+str(pee[0])
    X, y = prepare_data(dfr[region], 12)
    mont = 15
    X_train = X[:]
    y_train = y[:]
    n_features = 1
    X_train = X_train.reshape((X_train.shape[0], n_steps, n_features))
    model.fit(X_train, y_train, epochs=1, verbose=1)
    for k in pee:
        region = "Region "+str(k)
        regar = np.array(dfr[region])
        for i in range(15):
            X_test = regar[-12:].reshape((1, n_steps, n_features))
            yhat = model.predict(X_test, verbose=1)
            np.append(regar, yhat)
        fincsv[region] = regar[-15:]
        fincsv.to_csv("DeepSub.csv")
        print(k)

1/1 [==============================] - 0s 13ms/step
1
1/1 [==============================] - 0s 15ms/step
2
1/1 [==============================] - 0s 14ms/step
3
1/1 [==============================] - 0s 16ms/step
6
1/1 [==============================] - 0s 14ms/step
7
1/1 [==============================] - 0s 14ms/step
8
1/1 [==============================] - 0s 15ms/step
9
1/1 [==============================] - 0s 15ms/step
10
1/1 [==============================] - 0s 26ms/step
11
1/1 [==============================] - 0s 15ms/step
12
1/1 [==============================] - 0s 14ms/step
13
1/1 [==============================] - 0s 14ms/step
14
1/1 [==============================] - 0s 15ms/step
15
1/1 [==============================] - 0s 15ms/step
16
1/1 [==============================] - 0s 15ms/step
17
1/1 [==============================] - 0s 13ms/step
19
1/1 [==============================] - 0s 14ms/step
20
1/1 [==============================] - 0s 14ms/step
21
1/1 [============

Exception ignored in: <function ScopedTFGraph.__del__ at 0x000002D3F894E160>
Traceback (most recent call last):
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
AttributeError: deleter


1/1 [==============================] - 0s 15ms/step
28
1/1 [==============================] - 0s 17ms/step
30
1/1 [==============================] - 0s 16ms/step
31
1/1 [==============================] - 0s 15ms/step
32
1/1 [==============================] - 0s 15ms/step
33
1/1 [==============================] - 0s 17ms/step
34
1/1 [==============================] - 0s 13ms/step
35
1/1 [==============================] - 0s 14ms/step
38
1/1 [==============================] - 0s 15ms/step
39
1/1 [==============================] - 0s 15ms/step
41
1/1 [==============================] - 0s 15ms/step
42
1/1 [==============================] - 0s 15ms/step
43
1/1 [==============================] - 0s 21ms/step
50
1/1 [==============================] - 0s 16ms/step
51
1/1 [==============================] - 0s 20ms/step
52
1/1 [==============================] - 0s 17ms/step
53
1/1 [==============================] - 0s 17ms/step
62
1/1 [==============================] - 0s 14ms/step
71
1/1 [=====

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_21324/1578667902.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  fincsv[region] = regar[-15:]


1/1 [==============================] - 0s 15ms/step
2954
1/1 [==============================] - 0s 17ms/step
2955
1/1 [==============================] - 0s 16ms/step
2956
1/1 [==============================] - 0s 15ms/step
2974
1/1 [==============================] - 0s 14ms/step
3000
1/1 [==============================] - 0s 14ms/step
3003
1/1 [==============================] - 0s 13ms/step
3004
1/1 [==============================] - 0s 15ms/step
3029
1/1 [==============================] - 0s 14ms/step
3051
1/1 [==============================] - 0s 14ms/step
3058
1/1 [==============================] - 0s 14ms/step
3067
1/1 [==============================] - 0s 15ms/step
3075
1/1 [==============================] - 0s 15ms/step
3077
1/1 [==============================] - 0s 15ms/step
3078
1/1 [==============================] - 0s 17ms/step
3079
1/1 [==============================] - 0s 14ms/step
3080
1/1 [==============================] - 0s 18ms/step
3083
1/1 [==========================

KeyboardInterrupt: 

In [ ]:
fincsv

,Month,Region 2,Region 12
0,73,0.372061,0.373458
1,74,0.318767,0.314967
2,75,0.221102,0.245478
3,76,0.221851,0.230392
4,77,0.191496,0.173121
5,78,0.283412,0.254912
6,79,0.302669,0.268155
7,80,0.306597,0.282844
8,81,0.327412,0.313620
9,82,0.124563,0.204249
